### **1) REQUIREMENTS SETUP**### **1) REQUIREMENTS SETUP**# **Regression data preparation and modeling**

### **1) REQUIREMENTS SETUP**

In [ ]:
# !pip install -r requirements.txt

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from pathlib import Path
from linearmodels.panel import PanelOLS

### **2) MODULES IMPORT**

### **3) DATA Prep**

In [3]:
# Normalization of all the data used in the regression over comparable timeframe / format
data_fetcher_path = Path.cwd().parent / "data_fetcher"
dep_IPI = pd.read_csv(data_fetcher_path/"aggregate_df/EURO_indprod_dependent_df.csv")
dep_Stocks = pd.read_csv(data_fetcher_path/"aggregate_df/EURO_stock_dependent_df.csv")
#Other
exposure_df = pd.read_csv(data_fetcher_path/"country_tariff_exposure.csv")
openness_df = pd.read_csv(data_fetcher_path/"aggregate_df/EU_trade_openness_annual.csv")
controls_df = pd.read_csv(data_fetcher_path/"aggregate_df/country_specific_test_df.csv")

start_date = '2024-11'
end_date = '2025-08'

# IPI Dependent
time_period = pd.to_datetime(dep_IPI['Time'], errors='coerce').dt.to_period('M')
mask = (time_period >= pd.Period(start_date, 'M')) & (time_period <= pd.Period(end_date, 'M'))

cols = ['Country', 'Level 1 Index', 'Time', 'Indprod Index Value (I21)']
dep_IPI_filtered = dep_IPI.loc[mask, cols].copy()

# Ensure numeric for safe summation
dep_IPI_filtered['Indprod Index Value (I21)'] = pd.to_numeric(
    dep_IPI_filtered['Indprod Index Value (I21)'], errors='coerce'
)

# Collapse Level 1 Index: B/C -> "B+C", drop D, keep others (e.g., A) as-is
dep_IPI_filtered['Level 1 Index'] = (
    dep_IPI_filtered['Level 1 Index']
    .astype(str).str.strip()
    .replace({'B': 'B+C', 'C': 'B+C', 'D': np.nan})
)

# Drop rows mapped to NaN (i.e., D)
dep_IPI_filtered = dep_IPI_filtered.dropna(subset=['Level 1 Index'])

# Aggregate after collapsing (sum over Country, Time, collapsed Level 1 Index)
dep_IPI_collapsed = (
    dep_IPI_filtered
    .groupby(['Country', 'Time', 'Level 1 Index'], as_index=False, dropna=False)['Indprod Index Value (I21)']
    .sum()
)

# Sort & save
dep_IPI_collapsed = dep_IPI_collapsed.sort_values(['Country', 'Level 1 Index', 'Time']).reset_index(drop=True)
dep_IPI_collapsed.to_csv('data/dependent_variable/IndustrialProductionIndex_df.csv', index=False)

# Stocks Dependent
stock_period = pd.to_datetime(dep_Stocks['Time'], errors='coerce').dt.to_period('M')
stock_mask = (stock_period >= pd.Period(start_date, 'M')) & (stock_period <= pd.Period(end_date, 'M'))
stock_cols = ['Country', 'Stock Index', 'Time', 'Log Monthly Return', 'Volume']
dep_Stocks_filtered = dep_Stocks.loc[stock_mask, stock_cols].copy()
dep_Stocks_filtered = dep_Stocks_filtered.sort_values(['Country', 'Stock Index', 'Time']).reset_index(drop=True)
dep_Stocks_filtered.to_csv('data/dependent_variable/StockIndex_df.csv', index=False)

# Tariff(i,t) Independent
# ================================
# Tariff(i,t) Independent — FULL PANEL WITH ZERO-FILL
# ================================
from itertools import product

# 1. Round publication dates to month (same as before)
dt = pd.to_datetime(exposure_df['Publication_Date'], errors='coerce')
month_start = dt.dt.to_period('M').dt.start_time
next_month_start = (dt.dt.to_period('M') + 1).dt.start_time
delta_to_start = (dt - month_start).dt.days
delta_to_next = (next_month_start - dt).dt.days
rounded_month_start = pd.to_datetime(
    np.where(delta_to_start <= delta_to_next, month_start, next_month_start)
)

exposure_out = exposure_df.copy()
exposure_out['Time_dt'] = rounded_month_start
exposure_out['Time'] = exposure_out['Time_dt'].dt.strftime('%Y-%m')
exposure_out = exposure_out[['Country', 'Time', 'Exposure']].copy()

# 2. Define full panel: all countries × all months (2024-10 to 2025-08)
#     → 2024-10 needed for lag of 2024-11
all_countries = sorted(exposure_out['Country'].unique())
all_months = pd.date_range('2024-10', '2025-08', freq='MS').strftime('%Y-%m').tolist()

full_index = pd.MultiIndex.from_product([all_countries, all_months], names=['Country', 'Time'])
full_panel = pd.DataFrame(index=full_index).reset_index()

# 3. Merge observed shocks, fill missing with 0
exposure_full = full_panel.merge(exposure_out, on=['Country', 'Time'], how='left')
exposure_full['Exposure'] = exposure_full['Exposure'].fillna(0)

# 4. Sort and save
exposure_full = exposure_full.sort_values(['Country', 'Time']).reset_index(drop=True)
Path('data/independent_variable').mkdir(parents=True, exist_ok=True)
exposure_full.to_csv('data/independent_variable/CountryTariffExposure_df.csv', index=False)

# Openess(i,t) Independent
time_raw = openness_df['Time'].astype(str).str.strip()
year_num = pd.to_numeric(time_raw, errors='coerce')
year_dt = pd.to_datetime(time_raw, errors='coerce').dt.year
year = year_num.fillna(year_dt)
mask = year.isin([2024, 2025])
openness_out = openness_df.loc[mask].copy()
openness_out['Time'] = year.loc[mask].astype('Int64').astype(str)
openness_out = openness_out.sort_values(['Country', 'Time']).reset_index(drop=True)
openness_out.to_csv('data/transition_variable/TradeOpennessAnnual_df.csv', index=False)

# Controls (i,t)
controls_cols = [
    'Country',
    'Time',
    'GDP (Million USD)',
    'HICP (%, annual rate of change)',
    'Unemployment Rate (%pop in LF)'
]

controls_full = controls_df[controls_cols].copy()
controls_full['Time_dt'] = pd.to_datetime(controls_full['Time'], errors='coerce')
controls_full = controls_full.sort_values(['Country', 'Time_dt'])
controls_full['Unemployment Rate (%pop in LF)'] = (
    controls_full.groupby('Country')['Unemployment Rate (%pop in LF)'].diff()
)
controls_period = controls_full['Time_dt'].dt.to_period('M')
controls_mask = (controls_period >= pd.Period(start_date, 'M')) & (controls_period <= pd.Period(end_date, 'M'))
controls_out = controls_full.loc[controls_mask, [
    'Country',
    'Time_dt',
    'GDP (Million USD)',
    'HICP (%, annual rate of change)',
    'Unemployment Rate (%pop in LF)'
]].copy()
controls_out['Time'] = controls_out['Time_dt'].dt.strftime('%Y-%m')
controls_out = controls_out.drop(columns=['Time_dt'])
controls_out = controls_out.sort_values(['Country', 'Time']).reset_index(drop=True)
Path('data/control_variable').mkdir(parents=True, exist_ok=True)
controls_out.to_csv('data/control_variable/CountryControls_df.csv', index=False)



## 4) Baseline Simple OLS regressions

We estimate the **short-run effect** of U.S. tariff exposure on monthly macroeconomic outcomes (Industrial Production Index (B+C), Stocks Index Return, FX change).
The model includes **country** and **month fixed effects** to control for unobserved, time-invariant country characteristics and global shocks.

$$
y_{i,t} = \mu_i + \lambda_t
+ \alpha_0\,\text{Exposure}_{i,t}
+ \alpha_1\,\text{Exposure}_{i,t-1}
+ \Gamma' Z_{i,t}
+ \varepsilon_{i,t}
$$

**Where:**

- $y_{i,t}$ — Outcome variable (Industrial Production Index (B+C) YoY for country *i* in month *t*, Stocks Index for country *i* in month *t*, FX changes for country *i* in month *t*)
- $\text{Exposure}_{i,t}$ — Country *i*’s effective tariff exposure in month *t*
- $\text{Exposure}_{i,t-1}$ — One-month lag of tariff exposure (captures delayed reaction)
- $Z_{i,t}$ — Vector of monthly country-specific control variables (HICP YoY%, Δ unemployment)
- $\mu_i$ — Country fixed effects (absorbing structural country heterogeneity)
- $\lambda_t$ — Month fixed effects (absorbing global macro shocks)
- $\varepsilon_{i,t}$ — Idiosyncratic error term, clustered at the country level

---

### **Estimation Details**

- **Estimator:** OLS with two-way (country and month) fixed effects
- **Frequency:** Monthly (2024-10 → 2025-08)
- **Standard Errors:** Clustered by country
- **Sample:** EU countries only, using *effective* tariff exposure measure
- **Controls:** Domestic macro variables (HICP YoY%, Δ unemployment)

---

### **Parameters of Interest**

- $\alpha_0$ — Contemporaneous effect of tariff exposure
- $\alpha_1$ — One-month-lagged effect
- $\alpha_0 + \alpha_1$ — Cumulative 0–1-month effect (interpreted as the total short-run impact)


In [11]:

BASE = Path.cwd() / "data"

ipi      = pd.read_csv(BASE / "dependent_variable" / "IndustrialProductionIndex_df.csv")
stocks   = pd.read_csv(BASE / "dependent_variable" / "StockIndex_df.csv")
exposure = pd.read_csv(BASE / "independent_variable" / "CountryTariffExposure_df.csv")
controls = pd.read_csv(BASE / "control_variable" / "CountryControls_df.csv")


# Convert Time → Period[M] (keeps month arithmetic)

for df in (ipi, stocks, exposure, controls):
    df["Time"] = pd.to_datetime(df["Time"], errors="coerce").dt.to_period("M")

# Choose Y
# ---- Industrial Production (B+C) YoY ----
#df_dep = ipi.rename(columns={"Indprod Index Value (I21)": "y"})
# ---- Uncomment to run on Stock log-returns instead ----
df_dep = stocks.rename(columns={"Log Monthly Return": "y"})
#


exp = exposure.rename(columns={"Exposure": "Exposure_t"})
exp = exp.sort_values(["Country", "Time"])
exp["Exposure_t1"] = exp.groupby("Country")["Exposure_t"].shift(1)
exp = exp.dropna(subset=["Exposure_t1"])

df = (
    df_dep[["Country", "Time", "y"]]
    .merge(exp[["Country", "Time", "Exposure_t", "Exposure_t1"]],
           on=["Country", "Time"], how="inner")
    .merge(controls[["Country", "Time",
                     "HICP (%, annual rate of change)",
                     "Unemployment Rate (%pop in LF)"]],
           on=["Country", "Time"], how="inner")
)

df = df[(df["Time"] >= pd.Period("2024-11", "M")) &
        (df["Time"] <= pd.Period("2025-08", "M"))]

df["Time_dt"] = df["Time"].dt.start_time          # first day of month → datetime64
df = df.set_index(["Country", "Time_dt"])
df = df.drop(columns=["Time"])                    # optional cleanup

print("Months in final panel:", sorted(df.index.get_level_values(1).unique()))
print("Obs per country (min/avg/max):",
      df.groupby(level=0).size().min(),
      df.groupby(level=0).size().mean().round(1),
      df.groupby(level=0).size().max())


# Run two-way FE regression (cluster by country)

exog_vars = [
    "Exposure_t",
    "Exposure_t1",
    "HICP (%, annual rate of change)",
    "Unemployment Rate (%pop in LF)"
]

mod = PanelOLS(
    dependent=df["y"],
    exog=df[exog_vars],
    entity_effects=True,   # μ_i
    time_effects=True,     # λ_t
)

res = mod.fit(cov_type="clustered", cluster_entity=True)
print(res)

Months in final panel: [Timestamp('2024-11-01 00:00:00'), Timestamp('2024-12-01 00:00:00'), Timestamp('2025-01-01 00:00:00'), Timestamp('2025-02-01 00:00:00'), Timestamp('2025-03-01 00:00:00'), Timestamp('2025-04-01 00:00:00'), Timestamp('2025-05-01 00:00:00'), Timestamp('2025-06-01 00:00:00'), Timestamp('2025-07-01 00:00:00'), Timestamp('2025-08-01 00:00:00')]
Obs per country (min/avg/max): 10 10.0 10
                          PanelOLS Estimation Summary                           
Dep. Variable:                      y   R-squared:                        0.0394
Estimator:                   PanelOLS   R-squared (Between):             -0.2622
No. Observations:                 100   R-squared (Within):              -0.0110
Date:                Fri, Nov 07 2025   R-squared (Overall):             -0.0652
Time:                        01:19:20   Log-likelihood                    265.88
Cov. Estimator:             Clustered                                           
                           

## 5) Extended Specification — Linear Heterogeneity with Trade Openness Interaction

To allow the impact of tariff exposure to vary across countries with different levels of trade openness,
we extend the baseline specification by interacting exposure with lagged openness.
This captures whether **more open economies** react differently to tariff shocks.

$$
y_{i,t} = \mu_i + \lambda_t
+ \alpha_0\,\text{Exposure}_{i,t}
+ \alpha_1\,\text{Exposure}_{i,t-1}
+ \beta_0\,(\text{Exposure}_{i,t} \times \text{Openness}_{i,t-1})
+ \beta_1\,(\text{Exposure}_{i,t-1} \times \text{Openness}_{i,t-1})
+ \Gamma' Z_{i,t}
+ \varepsilon_{i,t}
$$

**Where:**

- $y_{i,t}$ — Outcome variable (e.g. Industrial Production YoY for country *i* in month *t*)
- $\text{Exposure}_{i,t}$ — Effective tariff exposure in month *t*
- $\text{Exposure}_{i,t-1}$ — One-month lag of exposure
- $\text{Openness}_{i,t-1}$ — Lagged trade openness index (annual, mapped to months)
- $(\text{Exposure}_{i,t} \times \text{Openness}_{i,t-1})$ — Interaction term capturing heterogeneous exposure effects
- $Z_{i,t}$ — Monthly domestic controls (HICP YoY, Δ unemployment)
- $\mu_i$ — Country fixed effects
- $\lambda_t$ — Month fixed effects
- $\varepsilon_{i,t}$ — Error term clustered by country


### **Interpretation**

- $\alpha_0$, $\alpha_1$ — Baseline (average) effects of exposure at zero openness
- $\beta_0$, $\beta_1$ — Marginal change in the exposure effect as openness increases
- **Marginal effect of exposure:**
  $$
  \frac{\partial y_{i,t}}{\partial \text{Exposure}_{i,t}}
  = \alpha_0 + \beta_0\,\text{Openness}_{i,t-1}
  $$
  and
  $$
  \frac{\partial y_{i,t}}{\partial \text{Exposure}_{i,t-1}}
  = \alpha_1 + \beta_1\,\text{Openness}_{i,t-1}
  $$
- Report **cumulative marginal effect** $(\alpha_0+\alpha_1) + (\beta_0+\beta_1)\text{Openness}_{i,t-1}$ evaluated at low, median, and high openness levels.

---

### **Estimation Details**

- **Estimator:** OLS with two-way (country and month) fixed effects
- **Frequency:** Monthly (2024-10 → 2025-09)
- **Standard Errors:** Clustered by country
- **Sample:** EU countries only
- **Controls:** Domestic macro variables (HICP YoY, unemployment)

---

### **Objective**

This model tests whether the **sensitivity to U.S. tariff shocks** depends on how open a country's economy is.
A positive $\beta_0$ or $\beta_1$ implies that **more open economies are more affected** by tariff changes,
while a negative coefficient indicates **buffering or diversification effects** from openness.


In [18]:
BASE = Path.cwd() / "data"

ipi        = pd.read_csv(BASE / "dependent_variable" / "IndustrialProductionIndex_df.csv")
stocks     = pd.read_csv(BASE / "dependent_variable" / "StockIndex_df.csv")
exposure   = pd.read_csv(BASE / "independent_variable" / "CountryTariffExposure_df.csv")
controls   = pd.read_csv(BASE / "control_variable" / "CountryControls_df.csv")
openness   = pd.read_csv(BASE / "transition_variable" / "TradeOpennessAnnual_df.csv")

for df in (ipi, stocks, exposure, controls):
    df["Time"] = pd.to_datetime(df["Time"], errors="coerce").dt.to_period("M")

openness = openness[["Country", "Time", "Trade_Openness_pct_GDP"]].copy()
openness.rename(columns={"Trade_Openness_pct_GDP": "Openness_annual"}, inplace=True)
openness["Year"] = pd.to_datetime(openness["Time"].astype(str), errors="coerce").dt.year
openness = openness.drop(columns=["Time"])

monthly_open = []
for year in [2024, 2025]:
    months = pd.date_range(f"{year}-01-01", f"{year}-12-31", freq="MS")
    temp   = pd.DataFrame({"Time_dt": months})
    temp["Time"] = temp["Time_dt"].dt.to_period("M")
    temp = temp.merge(openness[openness["Year"] == year], how="cross")
    monthly_open.append(temp)

openness_monthly = pd.concat(monthly_open, ignore_index=True)
openness_monthly = openness_monthly[["Country", "Time", "Openness_annual"]]
openness_monthly = openness_monthly.rename(columns={"Openness_annual": "Openness"})

df_dep = stocks.rename(columns={"Log Monthly Return": "y"})
# df_dep = ipi.rename(columns={"Indprod Index Value (I21)": "y"})

exp = exposure.rename(columns={"Exposure": "Exposure_t"})
exp = exp.sort_values(["Country", "Time"])
exp["Exposure_t1"] = exp.groupby("Country")["Exposure_t"].shift(1)

exp = exp.merge(openness_monthly, on=["Country", "Time"], how="left")
exp["Openness_t1"] = exp.groupby("Country")["Openness"].shift(1)

exp = exp.dropna(subset=["Exposure_t1", "Openness_t1"])

exp["Exp_t_x_Open_t1"]   = exp["Exposure_t"]   * exp["Openness_t1"]
exp["Exp_t1_x_Open_t1"]  = exp["Exposure_t1"]  * exp["Openness_t1"]

df = (
    df_dep[["Country", "Time", "y"]]
    .merge(exp[["Country", "Time",
                "Exposure_t", "Exposure_t1",
                "Exp_t_x_Open_t1", "Exp_t1_x_Open_t1",
                "Openness_t1"]],
           on=["Country", "Time"], how="inner")
    .merge(controls[["Country", "Time",
                     "HICP (%, annual rate of change)",
                     "Unemployment Rate (%pop in LF)"]],
           on=["Country", "Time"], how="inner")
)

df = df[(df["Time"] >= pd.Period("2024-11", "M")) &
        (df["Time"] <= pd.Period("2025-08", "M"))]

df["Time_dt"] = df["Time"].dt.start_time
df = df.set_index(["Country", "Time_dt"])
df = df.drop(columns=["Time"])

exog_vars = [
    "Exposure_t",
    "Exposure_t1",
    "Exp_t_x_Open_t1",
    "Exp_t1_x_Open_t1",
    "Openness_t1",
    "HICP (%, annual rate of change)",
    "Unemployment Rate (%pop in LF)"
]

mod = PanelOLS(
    dependent=df["y"],
    exog=df[exog_vars],
    entity_effects=True,
    time_effects=True,
)

res = mod.fit(cov_type="clustered", cluster_entity=True)
print(res)

print("\n=== MARGINAL EFFECTS OF TARIFF EXPOSURE ===")
open_levels = {
    "Low":     df["Openness_t1"].quantile(0.25),
    "Median":  df["Openness_t1"].median(),
    "High":    df["Openness_t1"].quantile(0.75)
}

for label, q in open_levels.items():
    me_t = res.params["Exposure_t"] + res.params["Exp_t_x_Open_t1"] * q
    var_t = (res.cov.loc["Exposure_t", "Exposure_t"] +
             2 * q * res.cov.loc["Exposure_t", "Exp_t_x_Open_t1"] +
             q**2 * res.cov.loc["Exp_t_x_Open_t1", "Exp_t_x_Open_t1"])
    se_t = np.sqrt(var_t)

    me_t1 = res.params["Exposure_t1"] + res.params["Exp_t1_x_Open_t1"] * q
    var_t1 = (res.cov.loc["Exposure_t1", "Exposure_t1"] +
              2 * q * res.cov.loc["Exposure_t1", "Exp_t1_x_Open_t1"] +
              q**2 * res.cov.loc["Exp_t1_x_Open_t1", "Exp_t1_x_Open_t1"])
    se_t1 = np.sqrt(var_t1)

    cov_cross = (res.cov.loc["Exposure_t", "Exposure_t1"] +
                 q * (res.cov.loc["Exposure_t", "Exp_t1_x_Open_t1"] +
                      res.cov.loc["Exp_t_x_Open_t1", "Exposure_t1"]) +
                 q**2 * res.cov.loc["Exp_t_x_Open_t1", "Exp_t1_x_Open_t1"])
    var_cum = var_t + var_t1 + 2 * cov_cross
    se_cum = np.sqrt(var_cum)
    me_cum = me_t + me_t1

    print(f"\n{label} Openness ({q:.4f}):")
    print(f"  Contemporaneous : {me_t: .5f} (se = {se_t: .5f})")
    print(f"  Lagged          : {me_t1:.5f} (se = {se_t1:.5f})")
    print(f"  Cumulative      : {me_cum:.5f} (se = {se_cum:.5f})")


                          PanelOLS Estimation Summary                           
Dep. Variable:                      y   R-squared:                        0.0430
Estimator:                   PanelOLS   R-squared (Between):             -0.4616
No. Observations:                 100   R-squared (Within):              -0.0054
Date:                Fri, Nov 07 2025   R-squared (Overall):             -0.1038
Time:                        02:07:44   Log-likelihood                    266.07
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.4753
Entities:                          10   P-value                           0.8495
Avg Obs:                      10.0000   Distribution:                    F(7,74)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             34.655
                            

## 6) Extended Specification — PSTR Model with Smooth Transition by Trade Openness

We now allow the effect of tariff exposure to vary **smoothly** with the level of trade openness.
Instead of assuming a linear interaction, we introduce a **logistic transition function** that captures gradual changes in the impact of exposure as openness increases.

The logistic transition function is defined as:

$$
G\!\left(\text{Openness}_{i,t-1};\gamma,c\right)
= \frac{1}{1 + \exp\!\big[-\gamma\big(\text{Openness}_{i,t-1} - c\big)\big]}.
$$

The corresponding PSTR regression is:

$$
\begin{aligned}
y_{i,t} &= \mu_i + \lambda_t
+ \alpha_0\,\text{Exposure}_{i,t}
+ \alpha_1\,\text{Exposure}_{i,t-1} \\
&\quad + \big(\beta_0\,\text{Exposure}_{i,t} + \beta_1\,\text{Exposure}_{i,t-1}\big)
\,G\!\left(\text{Openness}_{i,t-1};\gamma,c\right)
+ \Gamma' Z_{i,t}
+ \varepsilon_{i,t}
\end{aligned}
$$

**Where:**

- $y_{i,t}$ — Outcome variable (e.g. Industrial Production YoY for country *i* in month *t*)
- $\text{Exposure}_{i,t}$ — Effective tariff exposure in month *t*
- $\text{Exposure}_{i,t-1}$ — One-month lag of exposure
- $\text{Openness}_{i,t-1}$ — Lagged trade openness index (annual, mapped to months)
- $G(\text{Openness}_{i,t-1};\gamma,c)$ — Logistic transition function with:
  - $c$: threshold (location) where $G=0.5$
  - $\gamma$: smoothness parameter controlling how sharp the transition is
- $Z_{i,t}$ — Monthly domestic controls (HICP YoY, Δ unemployment)
- $\mu_i$ — Country fixed effects
- $\lambda_t$ — Month fixed effects
- $\varepsilon_{i,t}$ — Error term clustered by country

---

### **Interpretation**

- $\alpha_0$, $\alpha_1$ — Baseline exposure effects when openness is low ($G \approx 0$)
- $\beta_0$, $\beta_1$ — Incremental effects as openness increases ($G \to 1$)
- **Marginal effects of exposure:**
  $$
  \frac{\partial y_{i,t}}{\partial \text{Exposure}_{i,t}}
  = \alpha_0 + \beta_0\,G(\text{Openness}_{i,t-1};\gamma,c),
  $$
  $$
  \frac{\partial y_{i,t}}{\partial \text{Exposure}_{i,t-1}}
  = \alpha_1 + \beta_1\,G(\text{Openness}_{i,t-1};\gamma,c).
  $$
- **Cumulative short-run effect (0–1 month):**
  $$
  (\alpha_0 + \alpha_1) + (\beta_0 + \beta_1)\,G(\text{Openness}_{i,t-1};\gamma,c).
  $$
- When $\gamma$ is large, $G(\cdot)$ approximates a sharp threshold model; when small, the transition is smooth.

---

### **Estimation Details**

- **Estimator:** Nonlinear least squares with two-way (country and month) fixed effects
- **Frequency:** Monthly (2024-10 → 2025-09)
- **Standard Errors:** Clustered by country
- **Sample:** EU countries only
- **Controls:** Domestic macro variables (HICP YoY, unemployment)
- **Pre-processing:** Standardize openness before estimation; report the threshold $c$ in original units after transformation

---

### **Objective**

This model identifies whether the **effect of U.S. tariff shocks on economic outcomes** depends on how open each country is to international trade.
The logistic transition function captures a **nonlinear response** — for example, more open economies may only become significantly affected **after** crossing a critical openness threshold.


In [19]:
BASE = Path.cwd() / "data"

ipi        = pd.read_csv(BASE / "dependent_variable" / "IndustrialProductionIndex_df.csv")
stocks     = pd.read_csv(BASE / "dependent_variable" / "StockIndex_df.csv")
exposure   = pd.read_csv(BASE / "independent_variable" / "CountryTariffExposure_df.csv")
controls   = pd.read_csv(BASE / "control_variable" / "CountryControls_df.csv")
openness   = pd.read_csv(BASE / "transition_variable" / "TradeOpennessAnnual_df.csv")

for df in (ipi, stocks, exposure, controls):
    df["Time"] = pd.to_datetime(df["Time"], errors="coerce").dt.to_period("M")

openness = openness[["Country", "Time", "Trade_Openness_pct_GDP"]].copy()
openness.rename(columns={"Trade_Openness_pct_GDP": "Openness_annual"}, inplace=True)
openness["Year"] = pd.to_datetime(openness["Time"].astype(str), errors="coerce").dt.year
openness = openness.drop(columns=["Time"])

monthly_open = []
for year in [2024, 2025]:
    months = pd.date_range(f"{year}-01-01", f"{year}-12-31", freq="MS")
    tmp = pd.DataFrame({"Time_dt": months})
    tmp["Time"] = tmp["Time_dt"].dt.to_period("M")
    tmp = tmp.merge(openness[openness["Year"] == year][["Country", "Openness_annual"]], how="cross")
    monthly_open.append(tmp)

openness_monthly = pd.concat(monthly_open, ignore_index=True)
openness_monthly = openness_monthly[["Country", "Time", "Openness_annual"]]
openness_monthly = openness_monthly.rename(columns={"Openness_annual": "Openness"})

df_dep = stocks.rename(columns={"Log Monthly Return": "y"})
# df_dep = ipi.rename(columns={"Indprod Index Value (I21)": "y"})

exp = exposure.rename(columns={"Exposure": "Exposure_t"}).sort_values(["Country", "Time"])
exp["Exposure_t1"] = exp.groupby("Country")["Exposure_t"].shift(1)

exp = exp.merge(openness_monthly, on=["Country", "Time"], how="left")
exp["Openness_t1"] = exp.groupby("Country")["Openness"].shift(1)

exp = exp.dropna(subset=["Exposure_t1", "Openness_t1"])

df = (
    df_dep[["Country", "Time", "y"]]
    .merge(exp[["Country", "Time", "Exposure_t", "Exposure_t1", "Openness_t1"]],
           on=["Country", "Time"], how="inner")
    .merge(controls[["Country", "Time",
                     "HICP (%, annual rate of change)",
                     "Unemployment Rate (%pop in LF)"]],
           on=["Country", "Time"], how="inner")
)

df = df[(df["Time"] >= pd.Period("2024-11", "M")) &
        (df["Time"] <= pd.Period("2025-08", "M"))].copy()

df["Time_dt"] = df["Time"].dt.start_time
df = df.set_index(["Country", "Time_dt"]).sort_index()
df.drop(columns=["Time"], inplace=True)

open_mean = df["Openness_t1"].mean()
open_std  = df["Openness_t1"].std(ddof=0)
df["Open_std"] = (df["Openness_t1"] - open_mean) / open_std

df = df.rename(columns={
    "HICP (%, annual rate of change)": "HICP",
    "Unemployment Rate (%pop in LF)": "Unemp"
})

def twoway_within_transform(panel_df, cols):
    """
    Double-demean columns (entity & time FE).
    Returns a new DataFrame with *_wt (within-transformed) columns.
    """
    out = panel_df.copy()
    ent_means = out.groupby(level=0)[cols].transform("mean")
    time_means = out.groupby(level=1)[cols].transform("mean")
    overall_means = out[cols].mean()
    wt = out[cols] - ent_means - time_means + overall_means
    wt.columns = [c + "_wt" for c in cols]
    return wt

base_cols = ["y", "Exposure_t", "Exposure_t1", "HICP", "Unemp", "Open_std"]
wt = twoway_within_transform(df, base_cols)
for c in wt.columns:
    df[c] = wt[c]

def G_logistic(z, gamma, c):
    return 1.0 / (1.0 + np.exp(-gamma * (z - c)))

def build_X_within(gamma, c, work_df):
    """
    Build within-transformed X for given (γ,c):
      [ Exp_t_wt, Exp_t1_wt, (Exp_t*G)_wt, (Exp_t1*G)_wt, HICP_wt, Unemp_wt ]
    """
    z = work_df["Open_std_wt"].to_numpy()
    G = G_logistic(z, gamma, c)
    X = np.column_stack([
        work_df["Exposure_t_wt"].to_numpy(),
        work_df["Exposure_t1_wt"].to_numpy(),
        work_df["Exposure_t_wt"].to_numpy()  * G,
        work_df["Exposure_t1_wt"].to_numpy() * G,
        work_df["HICP_wt"].to_numpy(),
        work_df["Unemp_wt"].to_numpy(),
    ])
    return X

y_wt = df["y_wt"].to_numpy()

def ssr_objective(theta):
    gamma, c = theta
    if gamma <= 0:
        return 1e12 + (abs(gamma) + 1.0) * 1e12
    X = build_X_within(gamma, c, df)
    beta_hat, *_ = np.linalg.lstsq(X, y_wt, rcond=None)
    resid = y_wt - X @ beta_hat
    return float(resid @ resid)

theta0 = np.array([1.0, 0.0])
bounds = [(1e-3, 100.0), (-3.0, 3.0)]

opt = minimize(ssr_objective, theta0, method="L-BFGS-B", bounds=bounds)
gamma_hat, c_hat_std = opt.x
print("\n=== PSTR (NLS) transition estimates ===")
print(f"  gamma (smoothness): {gamma_hat: .4f}")
print(f"  c (threshold, standardized): {c_hat_std: .4f}")
c_hat_orig = open_mean + open_std * c_hat_std
print(f"  c (threshold, ORIGINAL openness units): {c_hat_orig: .4f}")

G_hat = G_logistic(df["Open_std"].to_numpy(), gamma_hat, c_hat_std)
df["G_hat"] = G_hat

df["Exp_t_G"]   = df["Exposure_t"]  * df["G_hat"]
df["Exp_t1_G"]  = df["Exposure_t1"] * df["G_hat"]

exog_cols = ["Exposure_t", "Exposure_t1", "Exp_t_G", "Exp_t1_G", "HICP", "Unemp"]

mod = PanelOLS(
    dependent=df["y"],
    exog=df[exog_cols],
    entity_effects=True,
    time_effects=True,
)
res = mod.fit(cov_type="clustered", cluster_entity=True)
print("\n=== PSTR Fixed-Effects (linear part) ===")
print(res)

print("\n=== MARGINAL EFFECTS OF TARIFF EXPOSURE (PSTR) ===")
open_levels = {
    "Low":    df["Openness_t1"].quantile(0.25),
    "Median": df["Openness_t1"].quantile(0.50),
    "High":   df["Openness_t1"].quantile(0.75),
}

alpha0 = res.params["Exposure_t"]
alpha1 = res.params["Exposure_t1"]
beta0  = res.params["Exp_t_G"]
beta1  = res.params["Exp_t1_G"]
V = res.cov

for label, q_orig in open_levels.items():
    z = (q_orig - open_mean) / open_std
    Gq = G_logistic(z, gamma_hat, c_hat_std)

    me_t = alpha0 + beta0 * Gq
    var_t = (
        V.loc["Exposure_t", "Exposure_t"]
        + 2*Gq*V.loc["Exposure_t", "Exp_t_G"]
        + (Gq**2)*V.loc["Exp_t_G", "Exp_t_G"]
    )
    se_t = float(np.sqrt(var_t))

    me_t1 = alpha1 + beta1 * Gq
    var_t1 = (
        V.loc["Exposure_t1", "Exposure_t1"]
        + 2*Gq*V.loc["Exposure_t1", "Exp_t1_G"]
        + (Gq**2)*V.loc["Exp_t1_G", "Exp_t1_G"]
    )
    se_t1 = float(np.sqrt(var_t1))

    cov_cross = (
        V.loc["Exposure_t", "Exposure_t1"]
        + Gq*(V.loc["Exposure_t", "Exp_t1_G"] + V.loc["Exp_t_G", "Exposure_t1"])
        + (Gq**2)*V.loc["Exp_t_G", "Exp_t1_G"]
    )
    var_cum = var_t + var_t1 + 2*cov_cross
    se_cum = float(np.sqrt(var_cum))
    me_cum = me_t + me_t1

    print(f"\n{label} Openness (orig={q_orig:.4f}, z={z:.3f}, G={Gq:.3f}):")
    print(f"  ∂y/∂Exposure_t      : {me_t: .6f}  (se = {se_t: .6f})")
    print(f"  ∂y/∂Exposure_(t-1)  : {me_t1:.6f}  (se = {se_t1:.6f})")
    print(f"  Cumulative (0–1 mo) : {me_cum:.6f}  (se = {se_cum:.6f})")



=== PSTR (NLS) transition estimates ===
  gamma (smoothness):  1.0000
  c (threshold, standardized):  0.0000
  c (threshold, ORIGINAL openness units):  5.1403

=== PSTR Fixed-Effects (linear part) ===
                          PanelOLS Estimation Summary                           
Dep. Variable:                      y   R-squared:                        0.0445
Estimator:                   PanelOLS   R-squared (Between):             -0.2374
No. Observations:                 100   R-squared (Within):              -0.0043
Date:                Fri, Nov 07 2025   R-squared (Overall):             -0.0546
Time:                        02:08:27   Log-likelihood                    266.15
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.5818
Entities:                          10   P-value                           0.7438
Avg Obs:                      10.0000   Distribution:                